In [27]:
#%matplotlib notebook
from matplotlib import pyplot as plt
#from matplotlib.patches import Polygon
#from defs_optique import dessine_rapporteur
from numpy import *
# ajout curseur et bouton
from ipywidgets import Button
from ipywidgets import interactive

n = 1.5                     # indice de réfraction
A = 60*pi/180               # angle d'ouverture du prisme
alpha1 = 30*pi/180          # pente du rayon incident par rapport à l'horizontale en rad
p = tan(A/2)                # on aura besoin de cette grandeur plusieurs fois
theta1 = alpha1 + A/2       # angle d'incidence sur face d'entrée
D = 0                       # initialisation angle déviation

def rayon(n, A, theta1):
    # calcul des points P
    theta1prime = arcsin(sin(theta1)/n)  # angle de réfraction
    alpha2 = theta1prime - A/2           # pente du rayon intérieur
    theta2prime =   alpha2 - A/2         # angle d'incidence sur face 2
    normale2 = True                      # test tracé de la normale du rayon sortant
    alpha1 = theta1 -A/2                 # angle incidentce par rapport à l'horizontal
    p = tan(A/2)                         # on aura besoin de cette grandeur plusieurs fois

    y1 = -0.5                                    # point P2
    x1 = y1*p                                    # point P2
    x0, y0 = x1 - cos(alpha1), y1 - sin(alpha1)  # point P1
    x = [x0, x1]                                 # rayon P1P2
    y = [y0, y1]                                 # rayon P1P2
    x5 = x1 + cos(alpha1)                        # point P5 pour direction rayon incident et définir déviation
    y5 = y1 + sin(alpha1)                        # point P5 pour direction rayon incident et définir déviation

    if (alpha2<A/2-pi/2):        # si le rayon intérieur ne ressort pas
        x += [x[1] + cos(alpha2)]
        y += [y[1] + sin(alpha2)]
        normale2 = False
    else:                        # si le rayon intérieur ressort
        t2 = tan(alpha2)
        x += [(x[1] * t2-y[1])/(1/p+t2)]         # point P3
        y += [y[1] + (x[2]-x[1])*t2]             # point P3
        # mettre le rayon réfléchi en cas de réflexion totale
        x6 = x[2] - 0.3*cos(alpha2-2*theta2prime)  # point P6 pour rayon réfléchi à revoir !
        y6 = y[2] - 0.3*sin(alpha2-2*theta2prime)  # point P6 pour rayon réfléchi

        if(abs(n*sin(theta2prime))<1):  # si pas de réflexion totale
            theta2 = arcsin(n*sin(theta2prime))
            alpha3 = theta2 + A/2
            x += [x[2] + cos(alpha3)]            # point P4
            y += [y[2] + sin(alpha3)]            # point P4
            normale2 = True
        else :
            normale2 = False


    # les angles
    #print(theta1*180/pi, theta1prime*180/pi, alpha2*180/pi, theta2prime*180/pi)
    # angle de déviation
    if normale2 == True :
        D = (alpha1 - alpha3)*180/pi
    else :
        D = 0
        
    return(x, y, normale2, D, x5, y5, x6, y6)

    
def update(angleA, indice, angle):
    # tracé du prisme et du rayon
    n = indice
    A = angleA*pi/180
    theta1 = angle*pi/180
    #print(n, A*180/pi, theta1*180/pi)
    (x, y, normale2, D, x5, y5, x6, y6) = rayon(n, A, theta1)
    fig, ax = plt.subplots(figsize=(10,5))
    ax.axis('equal')
    plt.xlim(-1.2,1.2)
    plt.ylim(-1.1,0.1)
    p = tan(A/2)
    # tracé du prisme en fonction de A à travers p=tan(A/2)
    prisme = ax.plot([0, -1*p, 1*p, 0],[0, -1, -1, 0], c ='b')
    # définition du rayon et des normales entrante et sortante
    line = ax.plot(x,y,c ='g')
    norm1 = ax.plot([x[1]+0.5*cos(A/2),x[1]-0.5*cos(A/2)], [y[1]-0.5*sin(A/2), y[1]+0.5*sin(A/2)], 
                    linestyle='dashed', c ='k')
    norm2 = ax.plot([x[2]+0.5*cos(A/2),x[2]-0.5*cos(A/2)], [y[2]+0.5*sin(A/2), y[2]-0.5*sin(A/2)], 
                    linestyle='dashed', c ='k')
    # prolongation rayon incident pour voir déviation
    prol = ax.plot([x[1], x5],[y[1], y5], linestyle='dashed', c ='r')
    # rayon réfléchi si réflexion totale
    reftot = ax.plot([x[2], x6],[y[2], y6], c ="g")
    # affichage de l'angle de déviation
    plt.text(0, 0.2, "Angle de déviation (°) =")
    dev = plt.text(0.7, 0.2, round(D,2))
    plt.show()


interactive_plot = interactive(update, angleA=(30, 70, 1), indice=(1.2, 1.8, .1), angle=(0, 90, 1))
interactive_plot


interactive(children=(IntSlider(value=50, description='angleA', max=70, min=30), FloatSlider(value=1.5, descri…